<a href="https://colab.research.google.com/github/pranitha2144/Internship-iith/blob/master/Multilabelcnnpractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.2.0


In [0]:
training_set = pd.read_csv("/content/drive/My Drive/catsanddogsdataset/train.csv")

training_imgs = ["{}.jpg".format(x) for x in list(training_set.id)]

training_labels_1 = list(training_set['class_name'])
training_labels_2 = list(training_set['breed'])
training_set = pd.DataFrame( {'Images': training_imgs,'Animal': training_labels_1, 'Breed' : training_labels_2})

#Changing the type  to str
training_set.Animal = training_set.Animal.astype(str)
training_set.Breed = training_set.Breed.astype(str)

In [0]:
training_set['New_class'] = training_set['Animal'] + training_set['Breed']

In [6]:
print(training_set.head())

         Images Animal Breed New_class
0  img_1313.jpg      1    11       111
1  img_0642.jpg      2    21       221
2  img_1450.jpg      1    11       111
3  img_2201.jpg      1    12       112
4  img_0964.jpg      2    21       221


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,
                                   rotation_range=40,
                                   )

train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="/content/drive/My Drive/catsanddogsdataset/images_train",x_col="Images",
                                        y_col="New_class",
                                        class_mode="categorical",
                                        target_size=(128,128),
                                        batch_size=32)

Found 6206 validated image filenames belonging to 10 classes.


In [0]:
classifier = Sequential([Conv2D(16,(3,3),activation='relu',input_shape=(128,128,3)),
                         MaxPooling2D(2,2),
                         Conv2D(32,(3,3),activation='relu'),
                         MaxPooling2D(2,2),
                         Conv2D(64,(3,3),activation='relu'),
                         MaxPooling2D(2,2),
                         Flatten(),
                         Dense(256,activation='relu'),
                         Dense(10,activation='softmax')



])

In [0]:
#First Convolutional layer
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))

In [0]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [0]:
#second Convolutional layer
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [0]:
#Flattening
classifier.add(Flatten())

In [0]:
#Hidden Layer
classifier.add(Dense(units = 64, activation = 'relu'))

#Output Layer
classifier.add(Dense(units = 10 , activation = 'softmax'))

In [0]:
classifier.compile(optimizer = 'RMSprop', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])



In [17]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 12544)            

In [18]:
classifier.fit(train_generator, epochs = 25, steps_per_epoch = 25 )

Epoch 1/25
25/25 [==============================] - 393s 16s/step - loss: 2.5814 - categorical_accuracy: 0.1125 - accuracy: 0.1125
Epoch 2/25
25/25 [==============================] - 323s 13s/step - loss: 2.2430 - categorical_accuracy: 0.1704 - accuracy: 0.1704
Epoch 3/25
25/25 [==============================] - 260s 10s/step - loss: 2.1205 - categorical_accuracy: 0.2325 - accuracy: 0.2325
Epoch 4/25
25/25 [==============================] - 237s 9s/step - loss: 1.9761 - categorical_accuracy: 0.2763 - accuracy: 0.2763
Epoch 5/25
25/25 [==============================] - 219s 9s/step - loss: 1.9636 - categorical_accuracy: 0.2937 - accuracy: 0.2937
Epoch 6/25
25/25 [==============================] - 175s 7s/step - loss: 1.9133 - categorical_accuracy: 0.3025 - accuracy: 0.3025
Epoch 7/25
25/25 [==============================] - 169s 7s/step - loss: 1.8487 - categorical_accuracy: 0.3275 - accuracy: 0.3275
Epoch 8/25
25/25 [==============================] - 156s 6s/step - loss: 1.8264 - categ

In [0]:
test_set = pd.read_csv("/content/drive/My Drive/catsanddogsdataset/test.csv")
test_imgs = ["/content/drive/My Drive/catsanddogsdataset/images_test/{}.jpg".format(x) for x in list(test_set.id)]
test_set = pd.DataFrame( {'Images': test_imgs })

In [20]:
classes = train_generator.class_indices
print(classes)

{'111': 0, '112': 1, '113': 2, '114': 3, '115': 4, '221': 5, '222': 6, '223': 7, '224': 8, '225': 9}


In [21]:
inverted_classes = dict(map(reversed, classes.items()))
print(inverted_classes)

{0: '111', 1: '112', 2: '113', 3: '114', 4: '115', 5: '221', 6: '222', 7: '223', 8: '224', 9: '225'}


In [0]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,128,128,3))
  img_class = np.argmax(classifier.predict(test_img), axis=-1)
  prediction = img_class[0]
  Y_pred.append(prediction)

In [24]:
print(Y_pred)

[9, 3, 5, 1, 1, 9, 8, 7, 8, 1, 5, 8, 8, 5, 1, 7, 5, 7, 1, 0, 7, 8, 7, 6, 5, 6, 7, 1, 1, 7, 9, 1, 5, 9, 7, 8, 7, 7, 7, 2, 0, 1, 0, 2, 5, 5, 6, 9, 8, 1, 1, 7, 6, 8, 7, 1, 5, 5, 1, 7, 9, 8, 5, 9, 6, 8, 7, 9, 5, 6, 7, 9, 6, 9, 8, 1, 1, 1, 5, 6, 6, 8, 5, 7, 2, 7, 5, 7, 3, 9, 7, 5, 5, 7, 1, 9, 8, 7, 5, 0, 9, 8, 5, 7, 5, 8, 5, 9, 2, 7, 5, 8, 9, 7, 7, 5, 7, 5, 7, 8, 7, 5, 8, 9, 1, 6, 8, 1, 9, 7, 1, 7, 9, 5, 5, 7, 8, 5, 0, 1, 7, 8, 1, 7, 8, 5, 8, 5, 1, 8, 7, 7, 7, 7, 5, 6, 9, 1, 5, 1, 7, 1, 9, 0, 0, 1, 5, 8, 1, 5, 9, 9, 7, 7, 7, 5, 1, 8, 0, 1, 1, 7, 9, 7, 0, 5, 7, 7, 8, 8, 9, 8, 5, 7, 6, 7, 5, 7, 1, 5, 1, 9, 9, 6, 7, 7, 9, 9, 5, 8, 0, 7, 9, 5, 5, 7, 8, 7, 7, 5, 9, 5, 9, 0, 7, 8, 5, 9, 9, 5, 7, 1, 5, 9, 5, 6, 9, 7, 5, 7, 5, 5, 1, 5, 5, 8, 6, 8, 1, 5, 6, 1, 7, 9, 9, 5, 7, 7, 5, 5, 5, 8, 1, 5, 7, 9, 7, 5, 0, 7, 9, 5, 8, 5, 5, 7, 5, 7, 5, 7, 9, 1, 5, 1, 5, 5, 5, 7, 9, 7, 7, 9, 7, 1, 7, 6, 7, 5, 7, 7, 7, 8, 5, 7, 8, 0, 1, 5, 9, 9, 9, 8, 5, 5, 9, 1, 1, 7, 8, 5, 1, 9, 5, 7, 7, 7, 1, 8, 7, 7, 7, 7, 7, 

In [0]:
prediction_classes = [ inverted_classes.get(item,item) for item in Y_pred ]

In [0]:
animal = []
breed = []
for i in prediction_classes:
  animal.append(i[0]) # First character = class_name/Animal
  breed.append(i[1:]) # Last 2 characters = breed/Breed

In [0]:
predictions = {}
predictions['class_name'] = animal
predictions['breed'] = breed

In [0]:
#Writing to excel
pd.DataFrame(predictions).to_excel("/content/drive/My Drive/catsanddogsdataset/sample_submission.xlsx", index = False)